In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import TimeseriesGenerator
import tensorflow as tf
from datetime import datetime

In [20]:
def convert_to_date(x):
    return datetime.strptime(x, '%Y %m %d')

In [31]:
data=pd.read_csv('twins_1992-1997.csv',parse_dates=[['year','month','day']],date_parser=convert_to_date,keep_date_col=True)

C:\Users\DELL\AppData\Local\Temp\ipykernel_24860\1545339538.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  data=pd.read_csv('twins_1992-1997.csv',parse_dates=[['year','month','day']],date_parser=convert_to_date,keep_date_col=True)


In [32]:
#data=pd.read_csv('twins_1992-1997.csv')

In [33]:
data.head()

,year_month_day,year,month,day,latitude,longitude,zon.winds,mer.winds,humidity,air temp.,s.s.temp.
0,1992-01-01,1992,1,1,5.01,-110.00,-3.3,3.6,NaN,26.39,27.40
1,1992-01-01,1992,1,1,1.98,-140.40,-4.4,4.1,NaN,27.75,28.27
2,1992-01-01,1992,1,1,5.00,-140.00,NaN,NaN,NaN,26.70,28.27
3,1992-01-01,1992,1,1,-2.00,156.00,9.6,4.3,NaN,27.77,29.39
4,1992-01-01,1992,1,1,8.04,-110.06,NaN,NaN,86.3,26.72,27.30


In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77506 entries, 0 to 77505
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   year_month_day  77506 non-null  datetime64[ns]
 1   year            77506 non-null  object        
 2   month           77506 non-null  object        
 3   day             77506 non-null  object        
 4   latitude        77506 non-null  float64       
 5   longitude       77506 non-null  float64       
 6   zon.winds       66895 non-null  float64       
 7   mer.winds       66895 non-null  float64       
 8   humidity        62674 non-null  float64       
 9   air temp.       72158 non-null  float64       
 10  s.s.temp.       77506 non-null  float64       
dtypes: datetime64[ns](1), float64(7), object(3)
memory usage: 6.5+ MB


In [121]:
data['humidity'].interpolate(inplace=True)
data['zon.winds'].fillna(np.random.normal(data['zon.winds'].mean(), data['zon.winds'].std()), inplace=True)
data['mer.winds'].fillna(np.random.normal(data['mer.winds'].mean(), data['mer.winds'].std()), inplace=True)
data['humidity'] = data['humidity'].fillna(data['humidity'].mean())
data = data.dropna(subset=['air temp.'])


In [123]:
data['year']=data['year'].astype(float)
data['month']=data['month'].astype(float)
data['day']=data['day'].astype(float)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 72158 entries, 0 to 77505
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   year_month_day  72158 non-null  datetime64[ns]
 1   year            72158 non-null  float64       
 2   month           72158 non-null  float64       
 3   day             72158 non-null  float64       
 4   latitude        72158 non-null  float64       
 5   longitude       72158 non-null  float64       
 6   zon.winds       72158 non-null  float64       
 7   mer.winds       72158 non-null  float64       
 8   humidity        72158 non-null  float64       
 9   air temp.       72158 non-null  float64       
 10  s.s.temp.       72158 non-null  float64       
dtypes: datetime64[ns](1), float64(10)
memory usage: 6.6 MB


In [63]:
data.count()

year_month_day    72158
year              72158
month             72158
day               72158
latitude          72158
longitude         72158
zon.winds         72158
mer.winds         72158
humidity          72158
air temp.         72158
s.s.temp.         72158
dtype: int64

In [125]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV


features = ['year', 'month', 'day', 'latitude', 'longitude', 'zon.winds', 'mer.winds', 'humidity', 'air temp.']
target = 's.s.temp.'


X = data[features]
y = data[target]
scaler = StandardScaler()

tss = TimeSeriesSplit(n_splits=3)
for train_index, test_index in tss.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]




X_train_scaled = scaler.fit_transform(X_train)

# Transform the testing data using the same scaler
X_test_scaled = scaler.transform(X_test)

# Initialize the Random Forest Regressor
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [10, 20],
    'min_samples_split': [10, 20],
    'min_samples_leaf': [4, 10]
}
# Initialize the Random Forest Regressor
rf_model = RandomForestRegressor(random_state=42)

# Perform Grid Search with Cross-Validation
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=tss, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)


# Get the best parameters
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

# Train the model on the scaled training data
rf_model.fit(X_train_scaled, y_train)

# Predict on the scaled testing data
y_pred = rf_model.predict(X_test_scaled)

# Calculate performance metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")


Best parameters: {'max_depth': 20, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}
Mean Squared Error: 0.21416302455402209
R-squared: 0.9572457527397458


In [126]:
y_test_pred = rf_model.predict(X_test_scaled)
y_train_pred = rf_model.predict(X_train_scaled)

In [127]:
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

In [128]:
print("Training Data Performance")
print(f"Mean Squared Error: {mse_train}")
print(f"R-squared: {r2_train}")

Training Data Performance
Mean Squared Error: 0.012496583210517553
R-squared: 0.9966690862168668


In [129]:
print("\nTesting Data Performance")
print(f"Mean Squared Error: {mse_test}")
print(f"R-squared: {r2_test}")


Testing Data Performance
Mean Squared Error: 0.21416302455402209
R-squared: 0.9572457527397458
